In [1]:
import dask.dataframe as dd
import pandas as pd
import glob
import os

In [2]:
bar_data_file = './data_cleaned/merged_cleaned_data.csv'

bar_df = dd.read_csv(bar_data_file)

bar_df.head()

,Datum,Product,Aantal * prijs
0,29-09-2022 18:54,Too late - Rush Hour,50.0
1,29-09-2022 18:54,Shot tequila,9.0
2,29-09-2022 18:54,Heineken,7.0
3,29-09-2022 18:54,Recycle munt,5.0
4,29-09-2022 18:54,Buitenlands mix,8.5


In [3]:
scan_data_file = './data_output/ticket_concat.csv'

scan_df = dd.read_csv(scan_data_file, usecols=['first_event_date_start', 'last_event_date_end'])
scan_df.head()

,first_event_date_start,last_event_date_end
0,2024-07-28 14:00:00,2024-07-28 23:00:00
1,2024-07-28 14:00:00,2024-07-28 23:00:00
2,2024-07-28 14:00:00,2024-07-28 23:00:00
3,2024-07-28 14:00:00,2024-07-28 23:00:00
4,2024-07-28 14:00:00,2024-07-28 23:00:00


In [4]:
bar_df['datum_datum'] = dd.to_datetime(bar_df['Datum'], format='%d-%m-%Y %H:%M', dayfirst=True).dt.date
scan_df['datum_datum'] = dd.to_datetime(scan_df['first_event_date_start']).dt.date

In [ ]:
merged_df = dd.merge(bar_df, scan_df, on='datum_datum')

In [6]:
merged_df.head(5, compute=True)

: 

In [7]:
try:
    # Save in a directory structure
    merged_df.to_parquet(
        'data_output/merged_scan_bar',
        engine='pyarrow',
        compression='snappy',
        write_index=False,
        partition_size='64MB'
    )
    print("Successfully saved merged data to parquet format")
    
    # To safely peek at the data, load just one partition
    first_partition = dd.read_parquet('data_output/merged_scan_bar').partitions[0]
    print("\nFirst few rows of first partition:")
    print(first_partition.head())
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

: 